In [2]:
import torch

pt_file = torch.load("./geometric_data_processed.pt")

# pt_file[0].org_edge_index = pt_file[0].edge_index
# pt_file[1]['org_edge_index'] = pt_file[1]['edge_index']
# torch.save(pt_file, "./geometric_data_processed copy.pt")

pt_file

/home/shpark/.conda/envs/lrgb/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(Data(edge_index=[2, 6393282], edge_attr=[6393282, 15], x=[4773974, 3], y=[15535, 10], org_edge_index=[2, 4773974]),
 defaultdict(dict,
             {'edge_index': tensor([      0,     328,    1246,  ..., 6392788, 6393108, 6393282]),
              'edge_attr': tensor([      0,     328,    1246,  ..., 6392788, 6393108, 6393282]),
              'x': tensor([      0,     244,     926,  ..., 4773604, 4773842, 4773974]),
              'y': tensor([    0,     1,     2,  ..., 15533, 15534, 15535])}))

In [3]:
import torch
from tqdm import tqdm

pt_file = torch.load("./geometric_data_processed org.pt")

x = pt_file[0].x
y = pt_file[0].y
x_index_slice = pt_file[1]['x']
edge_index = pt_file[0].edge_index
edge_attr = pt_file[0].edge_attr
edge_index_slice = pt_file[1]['edge_index']
numberOfGraphs = pt_file[0].y.shape[0]

# line graph node
lg_x = edge_attr
lg_x_slice = edge_index_slice

# line graph edge
lg_edge = []
lg_edge_attr = []
lg_edge_slice = [0]


# graphIdx = 0
for graphIdx in tqdm(range(numberOfGraphs)):
  graphX = x[x_index_slice[graphIdx]:x_index_slice[graphIdx+1], :]
  graphEdgeIdx = edge_index[:, edge_index_slice[graphIdx]:edge_index_slice[graphIdx+1]]
  graphEdgeAttr = edge_attr[edge_index_slice[graphIdx]:edge_index_slice[graphIdx+1], :]

  # NOTE: Making line graph edges
  linegraphNodeIdx = graphEdgeIdx.T
  linegraphEdgeIdxMask = torch.nonzero(
    (linegraphNodeIdx[:, 1, None] == linegraphNodeIdx[:, 0]) &
    (linegraphNodeIdx[:, 0, None] != linegraphNodeIdx[:, 1])
  )
  linegraphEdgeIdx = linegraphNodeIdx[linegraphEdgeIdxMask]
  linegraphEdgeAttr_node = graphX[linegraphEdgeIdx[:, 0, 1]]

  # TODO: get line graph edge from graph edge
  edgeStartIdx = linegraphEdgeIdx[:, 0]
  edgeEndIdx = linegraphEdgeIdx[:, 1]
  edgeStartMask = torch.where(torch.all(graphEdgeIdx.T[:, None]  == edgeStartIdx, dim=2))
  edgeStartAttr = graphEdgeAttr[edgeStartMask[0]]
  edgeEndMask = torch.where(torch.all(graphEdgeIdx.T[:, None]  == edgeEndIdx, dim=2))
  edgeEndAttr = graphEdgeAttr[edgeEndMask[0]]
  linegraphEdgeAttr_edge = torch.cat([edgeStartAttr, edgeEndAttr], dim=1)
  linegraphEdgeAttr = torch.cat([linegraphEdgeAttr_node, linegraphEdgeAttr_edge], dim=1)


  lg_edge.append(linegraphEdgeIdxMask.T)
  lg_edge_attr.append(linegraphEdgeAttr)
  lg_edge_slice.append(lg_edge_slice[-1]+linegraphEdgeIdxMask.shape[0])

# NOTE: Create tensors for line graph edge, edge_attr, edge_slice
lg_edge = torch.cat(lg_edge, dim=1)
lg_edge_attr = torch.cat(lg_edge_attr, dim=0)
lg_edge_slice = torch.tensor(lg_edge_slice, dtype=torch.int64)




100%|██████████| 15535/15535 [01:04<00:00, 242.47it/s]


In [4]:
print(lg_edge.shape)
print(lg_edge_attr.shape)
print(lg_edge_slice)
print(x_index_slice.shape)

torch.Size([2, 6393282])
torch.Size([6393282, 15])
tensor([      0,     328,    1246,  ..., 6392788, 6393108, 6393282])
torch.Size([15536])


In [6]:
# NOTE: create line graph dataset
pt_file = torch.load("./geometric_data_processed copy.pt")

pt_file[0].x = lg_x
pt_file[1]['x'] = lg_x_slice
pt_file[0].edge_index = lg_edge
pt_file[0].edge_attr = lg_edge_attr
pt_file[1]['edge_index'] = lg_edge_slice
pt_file[1]['edge_attr'] = lg_edge_slice

torch.save(pt_file, "./geometric_data_processed lg.pt")

In [7]:
pt_file = torch.load("./geometric_data_processed org.pt")
print(pt_file)
pt_file = torch.load("./geometric_data_processed.pt")
print(pt_file)


(Data(edge_index=[2, 4773974], edge_attr=[4773974, 3], x=[2344859, 9], y=[15535, 10]), defaultdict(<class 'dict'>, {'edge_index': tensor([      0,     244,     926,  ..., 4773604, 4773842, 4773974]), 'edge_attr': tensor([      0,     244,     926,  ..., 4773604, 4773842, 4773974]), 'x': tensor([      0,     119,     457,  ..., 2344679, 2344792, 2344859]), 'y': tensor([    0,     1,     2,  ..., 15533, 15534, 15535])}))
(Data(edge_index=[2, 6393282], edge_attr=[6393282, 15], x=[4773974, 3], y=[15535, 10], org_edge_index=[2, 4773974]), defaultdict(<class 'dict'>, {'edge_index': tensor([      0,     328,    1246,  ..., 6392788, 6393108, 6393282]), 'edge_attr': tensor([      0,     328,    1246,  ..., 6392788, 6393108, 6393282]), 'x': tensor([      0,     244,     926,  ..., 4773604, 4773842, 4773974]), 'y': tensor([    0,     1,     2,  ..., 15533, 15534, 15535])}))


In [9]:
pt_file = torch.load("./geometric_data_processed.pt")

x = pt_file[0].x
x.dtype

torch.int64